# VGG16 Model for Image Classification

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt

from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import cv2
import glob
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.python.keras import backend as K
import plotly.graph_objects as go
import plotly.offline as py
autosize =False

from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline

In [5]:
import pandas as pd

In [6]:
train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
test=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')


# Modelling - VGG16 (Transfer Learning)

In [2]:
x1 = "128"
x2 = "128"
x_train_dim = "x_train_" + x1
x_test_dim = "x_test_" + x1

In [3]:
x_train = np.load('../input/siimisic-melanoma-resized-images/' + x_train_dim + ".npy")
x_train.shape

(33126, 128, 128, 3)

## Adding augmented numpy arrays for training set

In [34]:
arr = np.load('/kaggle/input/augmentednpy/augmentednpy/augmented.npy')
arr.shape

(1697, 128, 128, 3)

In [36]:
x_train = np.insert(x_train, x_train.shape[0], arr, axis=0)

In [6]:
x_train.shape

(33126, 128, 128, 3)

In [38]:
augmented = pd.Series([1 for i in range(1697)])
augmented.shape

(1697,)

## Getting the labels

In [4]:
y = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv').target
# y = y.append(augmented)
y.shape

(33126,)

In [40]:
y = y.append(augmented)
y.shape

(34823,)

## Getting class weights

In [11]:
from sklearn.utils import class_weight
 
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y),
                                                 y)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass classes=[0 1], y=0        0
1        0
2        0
3        0
4        0
        ..
33121    0
33122    0
33123    0
33124    0
33125    0
Name: target, Length: 33126, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error



In [12]:
class_weights = dict(enumerate(class_weights))

In [13]:
class_weights

{0: 0.5089730194825149, 1: 28.361301369863014}

## Splitting the dataset

In [6]:
from sklearn.model_selection import train_test_split
train_imgs, validation_imgs, y_train, y_val = train_test_split(x_train,y, test_size=0.2, random_state=1234)
train_imgs, test_imgs, y_train, y_test = train_test_split(train_imgs,y_train, test_size=0.1, random_state=1234)

In [7]:
test_imgs.shape

(2650, 128, 128, 3)

In [26]:
import gc
del x_train
gc.collect()

30

In [17]:
# define parameters for model training
batch_size = 128
num_classes = 2
epochs = 50
input_shape = (128,128,3)

In [18]:
# we will use Adam optimizer
opt = Adam(lr=1e-5)

nb_train_steps = train_imgs.shape[0]//batch_size
nb_val_steps=validation_imgs.shape[0]//batch_size

print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

Number of training and validation steps: 186 and 46


## Type 2 Data Augmentation
ImageDataGenerator class ensures that the model receives new variations of the images at each epoch.

In [19]:
# Pixel Normalization and Image Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

# no need to create augmentation images for validation data, only rescaling the pixels
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_imgs, y_train, batch_size=32)
val_generator = val_datagen.flow(validation_imgs, y_val, batch_size=32)

In [49]:
del train_imgs,validation_imgs
gc.collect()

NameError: name 'gc' is not defined

## Activating the trainable layers in a VGG16 model

In [20]:
from keras.applications import vgg16
from keras.models import Model
import keras

vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

58892288/58889256 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f6bfa354e10>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bfa3809d0>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bfaaed450>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6c078b5510>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb3cf2f50>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bfa380b10>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6bb014cf10>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb014c910>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb0147d50>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb0157250>,block3_conv3,False


In [21]:
vgg_model.trainable = True

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f6bfa354e10>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bfa3809d0>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bfaaed450>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6c078b5510>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb3cf2f50>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bfa380b10>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f6bb014cf10>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb014c910>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb0147d50>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6bb0157250>,block3_conv3,False


## Adding layers for classification

In [22]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()],optimizer=opt)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 8192)              14714688  
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 19,172,673
Trainable params: 17,437,185
Non-trainable params: 1,735,488
____________________________________

## Training

In [24]:
model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=val_generator, validation_steps=nb_val_steps,class_weight=class_weights, 
                              verbose=1)

Epoch 1/50
186/186 [==============================] - 25s 134ms/step - loss: 0.7442 - auc: 0.5234 - val_loss: 0.8424 - val_auc: 0.6982
Epoch 2/50
186/186 [==============================] - 24s 130ms/step - loss: 0.6425 - auc: 0.6263 - val_loss: 1.0071 - val_auc: 0.6936
Epoch 3/50
186/186 [==============================] - 24s 129ms/step - loss: 0.6120 - auc: 0.7113 - val_loss: 0.5946 - val_auc: 0.8077
Epoch 4/50
186/186 [==============================] - 23s 126ms/step - loss: 0.6222 - auc: 0.7231 - val_loss: 0.7040 - val_auc: 0.7819
Epoch 5/50
186/186 [==============================] - 24s 129ms/step - loss: 0.5807 - auc: 0.7858 - val_loss: 0.5228 - val_auc: 0.8293
Epoch 6/50
186/186 [==============================] - 24s 131ms/step - loss: 0.5966 - auc: 0.7508 - val_loss: 0.3241 - val_auc: 0.8510
Epoch 7/50
186/186 [==============================] - 24s 129ms/step - loss: 0.5755 - auc: 0.7427 - val_loss: 0.3177 - val_auc: 0.7884
Epoch 8/50
186/186 [==============================] - 2

## Saving and Loading Model

In [25]:
import tensorflow as tf
model.save('vggmodel.h5')

In [8]:
model = tf.keras.models.load_model('/kaggle/input/trainedvgg16model/model/vggmodel.h5')

# Testing the model

In [26]:
del train_generator,val_generator
gc.collect()

NameError: name 'gc' is not defined

### Creating a function to return probability of having the cancer

In [9]:
from numpy import expand_dims
def tta_prediction(datagen, model, image, n_examples):
    # convert image into dataset
    samples = expand_dims(image, 0)
    # prepare iterator
    it = datagen.flow(samples, batch_size=n_examples)
    # make predictions for each augmented image
    probs = model.predict_generator(it, steps=n_examples, verbose=0)
    #print(len(probs))    
    prob = np.mean(probs, axis=1)    
    return prob

In [10]:
from tqdm import tqdm
# configure image data augmentation
test_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

In [11]:
test_imgs.shape

(2650, 128, 128, 3)

## Getting predictions for test samples

In [12]:
# test file
target=[]
#i = 0
for img in tqdm(test_imgs):
    prediction=tta_prediction(test_datagen,model,img,32)
    pred_y = prediction[0]
    val=0
    if pred_y>0.5:
        val=1
    target.append(val)

100%|██████████| 2650/2650 [01:56<00:00, 22.73it/s]


## Manual Checking of predictions of the model

In [14]:
siim = ['ISIC_0149568', 'ISIC_0188432', 'ISIC_0207268', 'ISIC_0232101', 'ISIC_0247330', 'ISIC_0250839', 'ISIC_0272509', 'ISIC_0273046', 'ISIC_0274382', 'ISIC_0275828', 'ISIC_0280912', 'ISIC_0286360', 'ISIC_0327406', 'ISIC_0333091', 'ISIC_0337631', 'ISIC_0351666', 'ISIC_0369831', 'ISIC_0489267', 'ISIC_0502582', 'ISIC_0504165', 'ISIC_0528044', 'ISIC_0533122', 'ISIC_0559335', 'ISIC_0572205', 'ISIC_0599605', 'ISIC_0624498', 'ISIC_0639769', 'ISIC_0645454', 'ISIC_0647224', 'ISIC_0662000', 'ISIC_0744013', 'ISIC_0779920', 'ISIC_0833889', 'ISIC_0844312', 'ISIC_0862745', 'ISIC_0911264', 'ISIC_0914168', 'ISIC_0924427', 'ISIC_0946787', 'ISIC_0952472', 'ISIC_0953317', 'ISIC_0961235', 'ISIC_0981639', 'ISIC_0986600', 'ISIC_1009042', 'ISIC_1037315', 'ISIC_1068686', 'ISIC_1071355', 'ISIC_1110583', 'ISIC_1113490', 'ISIC_1116483', 'ISIC_1124434', 'ISIC_1132496', 'ISIC_1157032', 'ISIC_1159891', 'ISIC_1162337', 'ISIC_1177153', 'ISIC_1177505', 'ISIC_1200192', 'ISIC_1210011', 'ISIC_1231419', 'ISIC_1241817', 'ISIC_1247683', 'ISIC_1255336', 'ISIC_1330008', 'ISIC_1356715', 'ISIC_1369967', 'ISIC_1375476', 'ISIC_1388552', 'ISIC_1393771', 'ISIC_1412843', 'ISIC_1440985', 'ISIC_1460932', 'ISIC_1473522', 'ISIC_1474461', 'ISIC_1484659', 'ISIC_1566428', 'ISIC_1569119', 'ISIC_1589997', 'ISIC_1605629', 'ISIC_1616031', 'ISIC_1620401', 'ISIC_1630301', 'ISIC_1654342', 'ISIC_1684127', 'ISIC_1698942', 'ISIC_1706912', 'ISIC_1728564', 'ISIC_1729223', 'ISIC_1752943', 'ISIC_1785627', 'ISIC_1797795', 'ISIC_1834920', 'ISIC_1866770', 'ISIC_1870415', 'ISIC_1881133', 'ISIC_1948545', 'ISIC_1952749', 'ISIC_1956504', 'ISIC_1959939', 'ISIC_1968421', 'ISIC_1975042', 'ISIC_2027582', 'ISIC_2033237', 'ISIC_2071966', 'ISIC_2072219', 'ISIC_2077116', 'ISIC_2101624', 'ISIC_2105563', 'ISIC_2141237', 'ISIC_2153401', 'ISIC_2153686', 'ISIC_2176112', 'ISIC_2182856', 'ISIC_2200197', 'ISIC_2212473', 'ISIC_2221217', 'ISIC_2232442', 'ISIC_2255411', 'ISIC_2268341', 'ISIC_2272218', 'ISIC_2306349', 'ISIC_2309744', 'ISIC_2311665', 'ISIC_2320481', 'ISIC_2337907', 'ISIC_2341875', 'ISIC_2342769', 'ISIC_2351999', 'ISIC_2358454', 'ISIC_2364150', 'ISIC_2406492', 'ISIC_2408815', 'ISIC_2412113', 'ISIC_2437633', 'ISIC_2447578', 'ISIC_2448123', 'ISIC_2481044', 'ISIC_2525084', 'ISIC_2535541', 'ISIC_2565206', 'ISIC_2607827', 'ISIC_2609984', 'ISIC_2637608', 'ISIC_2646829', 'ISIC_2668481', 'ISIC_2685593', 'ISIC_2697083', 'ISIC_2700320', 'ISIC_2720971', 'ISIC_2724529', 'ISIC_2739449', 'ISIC_2750230', 'ISIC_2757355', 'ISIC_2776906', 'ISIC_2787702', 'ISIC_2797353', 'ISIC_2822718', 'ISIC_2823287', 'ISIC_2837581', 'ISIC_2844539', 'ISIC_2871230', 'ISIC_2895555', 'ISIC_2937642', 'ISIC_2957732', 'ISIC_2967024', 'ISIC_2974214', 'ISIC_2975580', 'ISIC_2979268', 'ISIC_2987613', 'ISIC_3002956', 'ISIC_3009877', 'ISIC_3023646', 'ISIC_3025889', 'ISIC_3028754', 'ISIC_3065032', 'ISIC_3068885', 'ISIC_3084312', 'ISIC_3093026', 'ISIC_3097354', 'ISIC_3144222', 'ISIC_3159924', 'ISIC_3203344', 'ISIC_3207651', 'ISIC_3241636', 'ISIC_3244067', 'ISIC_3253484', 'ISIC_3286186', 'ISIC_3301933', 'ISIC_3319229', 'ISIC_3338629', 'ISIC_3341710', 'ISIC_3343475', 'ISIC_3359880', 'ISIC_3372750', 'ISIC_3391427', 'ISIC_3394520', 'ISIC_3399306', 'ISIC_3408231', 'ISIC_3416107', 'ISIC_3435596', 'ISIC_3436033', 'ISIC_3440759', 'ISIC_3446357', 'ISIC_3464344', 'ISIC_3475660', 'ISIC_3517311', 'ISIC_3561065', 'ISIC_3590167', 'ISIC_3673080', 'ISIC_3684488', 'ISIC_3696488', 'ISIC_3697702', 'ISIC_3733653', 'ISIC_3738751', 'ISIC_3775417', 'ISIC_3778192', 'ISIC_3817719', 'ISIC_3821242', 'ISIC_3824206', 'ISIC_3909039', 'ISIC_3915890', 'ISIC_3936223', 'ISIC_3941123', 'ISIC_3944634', 'ISIC_3963183', 'ISIC_3965087', 'ISIC_3969411', 'ISIC_3976496', 'ISIC_3979826', 'ISIC_3985031', 'ISIC_3993924', 'ISIC_3996796', 'ISIC_4000481', 'ISIC_4009870', 'ISIC_4012047', 'ISIC_4120174', 'ISIC_4163686', 'ISIC_4170698', 'ISIC_4223155', 'ISIC_4229473', 'ISIC_4230049', 'ISIC_4266533', 'ISIC_4276255', 'ISIC_4346693', 'ISIC_4362713', 'ISIC_4377208', 'ISIC_4378189', 'ISIC_4398011', 'ISIC_4404772', 'ISIC_4408559', 'ISIC_4425322', 'ISIC_4425657', 'ISIC_4439054', 'ISIC_4469476', 'ISIC_4493482', 'ISIC_4508878', 'ISIC_4517738', 'ISIC_4523238', 'ISIC_4523382', 'ISIC_4525317', 'ISIC_4584265', 'ISIC_4619789', 'ISIC_4619903', 'ISIC_4649440', 'ISIC_4653503', 'ISIC_4676270', 'ISIC_4730066', 'ISIC_4810142', 'ISIC_4842587', 'ISIC_4851366', 'ISIC_4874581', 'ISIC_4916704', 'ISIC_4936662', 'ISIC_4938994', 'ISIC_4960784', 'ISIC_4960802', 'ISIC_4967564', 'ISIC_4967586', 'ISIC_4980656', 'ISIC_5021861', 'ISIC_5037784', 'ISIC_5044438', 'ISIC_5045600', 'ISIC_5046082', 'ISIC_5084113', 'ISIC_5086349', 'ISIC_5137933', 'ISIC_5140952', 'ISIC_5147292', 'ISIC_5166798', 'ISIC_5167209', 'ISIC_5191344', 'ISIC_5216586', 'ISIC_5247508', 'ISIC_5247560', 'ISIC_5258657', 'ISIC_5260050', 'ISIC_5267208', 'ISIC_5272223', 'ISIC_5315062', 'ISIC_5331102', 'ISIC_5343231', 'ISIC_5365617', 'ISIC_5369000', 'ISIC_5388184', 'ISIC_5391758', 'ISIC_5415761', 'ISIC_5425721', 'ISIC_5434661', 'ISIC_5495413', 'ISIC_5527224', 'ISIC_5533992', 'ISIC_5561003', 'ISIC_5576354', 'ISIC_5581297', 'ISIC_5581931', 'ISIC_5602826', 'ISIC_5642824', 'ISIC_5667461', 'ISIC_5673518', 'ISIC_5714411', 'ISIC_5732201', 'ISIC_5733748', 'ISIC_5747486', 'ISIC_5747942', 'ISIC_5758388', 'ISIC_5797062', 'ISIC_5829938', 'ISIC_5867911', 'ISIC_5868291', 'ISIC_5869480', 'ISIC_5891384', 'ISIC_5922588', 'ISIC_5983500', 'ISIC_6017985', 'ISIC_6024335', 'ISIC_6056121', 'ISIC_6089318', 'ISIC_6110888', 'ISIC_6124481', 'ISIC_6125652', 'ISIC_6220625', 'ISIC_6228963', 'ISIC_6248008', 'ISIC_6251753', 'ISIC_6255113', 'ISIC_6269166', 'ISIC_6278273', 'ISIC_6285925', 'ISIC_6303107', 'ISIC_6313487', 'ISIC_6348638', 'ISIC_6352565', 'ISIC_6355353', 'ISIC_6392461', 'ISIC_6398283', 'ISIC_6398669', 'ISIC_6404049', 'ISIC_6450034', 'ISIC_6496251', 'ISIC_6496370', 'ISIC_6509682', 'ISIC_6512653', 'ISIC_6516710', 'ISIC_6526489', 'ISIC_6535558', 'ISIC_6539127', 'ISIC_6541615', 'ISIC_6555290', 'ISIC_6561111', 'ISIC_6581411', 'ISIC_6584579', 'ISIC_6613150', 'ISIC_6620377', 'ISIC_6625102', 'ISIC_6634951', 'ISIC_6671773', 'ISIC_6688417', 'ISIC_6692344', 'ISIC_6695831', 'ISIC_6713432', 'ISIC_6714082', 'ISIC_6767569', 'ISIC_6784593', 'ISIC_6785680', 'ISIC_6786198', 'ISIC_6809586', 'ISIC_6829008', 'ISIC_6841294', 'ISIC_6851408', 'ISIC_6904082', 'ISIC_6927840', 'ISIC_6931277', 'ISIC_6945634', 'ISIC_6961503', 'ISIC_6966001', 'ISIC_6971606', 'ISIC_6982386', 'ISIC_7035246', 'ISIC_7070020', 'ISIC_7074321', 'ISIC_7075474', 'ISIC_7079349', 'ISIC_7084553', 'ISIC_7098141', 'ISIC_7100755', 'ISIC_7101732', 'ISIC_7130929', 'ISIC_7181296', 'ISIC_7183212', 'ISIC_7190243', 'ISIC_7205745', 'ISIC_7218890', 'ISIC_7233368', 'ISIC_7242384', 'ISIC_7253401', 'ISIC_7261174', 'ISIC_7265673', 'ISIC_7287303', 'ISIC_7293518', 'ISIC_7295035', 'ISIC_7312977', 'ISIC_7320371', 'ISIC_7327423', 'ISIC_7346228', 'ISIC_7346739', 'ISIC_7349772', 'ISIC_7371801', 'ISIC_7372719', 'ISIC_7410175', 'ISIC_7460887', 'ISIC_7466678', 'ISIC_7489564', 'ISIC_7503950', 'ISIC_7520843', 'ISIC_7536704', 'ISIC_7546219', 'ISIC_7577395', 'ISIC_7612992', 'ISIC_7614221', 'ISIC_7626256', 'ISIC_7658729', 'ISIC_7662910', 'ISIC_7680948', 'ISIC_7693611', 'ISIC_7724762', 'ISIC_7730937', 'ISIC_7748311', 'ISIC_7763042', 'ISIC_7785723', 'ISIC_7788318', 'ISIC_7800750', 'ISIC_7844825', 'ISIC_7872595', 'ISIC_7897925', 'ISIC_7914631', 'ISIC_7927525', 'ISIC_7936755', 'ISIC_7939816', 'ISIC_7948290', 'ISIC_7960321', 'ISIC_7960869', 'ISIC_7961365', 'ISIC_7973598', 'ISIC_8002875', 'ISIC_8006211', 'ISIC_8009536', 'ISIC_8011058', 'ISIC_8027899', 'ISIC_8040024', 'ISIC_8052769', 'ISIC_8066110', 'ISIC_8068693', 'ISIC_8106899', 'ISIC_8124900', 'ISIC_8133350', 'ISIC_8145348', 'ISIC_8157654', 'ISIC_8167374', 'ISIC_8175453', 'ISIC_8194835', 'ISIC_8207974', 'ISIC_8230318', 'ISIC_8231999', 'ISIC_8249854', 'ISIC_8252406', 'ISIC_8263489', 'ISIC_8281208', 'ISIC_8306697', 'ISIC_8325226', 'ISIC_8355446', 'ISIC_8379956', 'ISIC_8381531', 'ISIC_8397285', 'ISIC_8414609', 'ISIC_8417873', 'ISIC_8437493', 'ISIC_8450656', 'ISIC_8483382', 'ISIC_8485716', 'ISIC_8493108', 'ISIC_8496052', 'ISIC_8497457', 'ISIC_8509577', 'ISIC_8521610', 'ISIC_8527092', 'ISIC_8545949', 'ISIC_8564474', 'ISIC_8566006', 'ISIC_8609756', 'ISIC_8615491', 'ISIC_8617277', 'ISIC_8621568', 'ISIC_8672524', 'ISIC_8677053', 'ISIC_8692218', 'ISIC_8702237', 'ISIC_8724944', 'ISIC_8732757', 'ISIC_8737794', 'ISIC_8789051', 'ISIC_8838753', 'ISIC_8859274', 'ISIC_8866651', 'ISIC_8872158', 'ISIC_8882374', 'ISIC_8892236', 'ISIC_8922893', 'ISIC_8951909', 'ISIC_8985109', 'ISIC_9000234', 'ISIC_9000824', 'ISIC_9001948', 'ISIC_9012513', 'ISIC_9022005', 'ISIC_9038318', 'ISIC_9048507', 'ISIC_9052500', 'ISIC_9053910', 'ISIC_9124447', 'ISIC_9130157', 'ISIC_9164054', 'ISIC_9174306', 'ISIC_9193127', 'ISIC_9199429', 'ISIC_9210255', 'ISIC_9228573', 'ISIC_9268487', 'ISIC_9286292', 'ISIC_9312164', 'ISIC_9317483', 'ISIC_9351516', 'ISIC_9367522', 'ISIC_9402324', 'ISIC_9509757', 'ISIC_9544832', 'ISIC_9547725', 'ISIC_9548821', 'ISIC_9556334', 'ISIC_9571842', 'ISIC_9579703', 'ISIC_9583969', 'ISIC_9585329', 'ISIC_9593940', 'ISIC_9596721', 'ISIC_9612481', 'ISIC_9626843', 'ISIC_9677503', 'ISIC_9685931', 'ISIC_9754098', 'ISIC_9770677', 'ISIC_9785077', 'ISIC_9821002', 'ISIC_9828846', 'ISIC_9863642', 'ISIC_9870885', 'ISIC_9900191', 'ISIC_9910791', 'ISIC_9955163', 'ISIC_9963177', 'ISIC_9967383', 'ISIC_9978107', 'ISIC_9998682']

In [15]:
truelis = []
for i in tqdm(siim):
    prediction = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train/"+i+'.jpg'
    # prediction ='/kaggle/input/augmenteddataset/augmentfiles/aug_image_ISIC_0024315.png'
    img = cv2.imread(prediction)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    truelis.append(tta_prediction(test_datagen,model,res,1)>0.5)

100%|██████████| 584/584 [01:24<00:00,  6.90it/s]


In [16]:
truelis.count(True)

471

## Generating Classification Report

In [13]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test, target))
print(classification_report(y_test, target))

[[1914  686]
 [   6   44]]
              precision    recall  f1-score   support

           0       1.00      0.74      0.85      2600
           1       0.06      0.88      0.11        50

    accuracy                           0.74      2650
   macro avg       0.53      0.81      0.48      2650
weighted avg       0.98      0.74      0.83      2650



# Creating a numpy file for the augmented images

In [12]:
import os
path= '/kaggle/input/augmenteddataset/augmentfiles'
dir_list = os.listdir(path)
print(dir_list[:5])

['aug_image_ISIC_0033856.png', 'aug_image_ISIC_0032424.png', 'aug_image_ISIC_5260050.png', 'aug_image_ISIC_7261174.png', 'aug_image_ISIC_0028548.png']


In [13]:
from PIL import Image
from tqdm import tqdm
imSize=128
def preprocess_image(image_path, desired_size=imSize):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    return res

In [32]:
arr = np.array([np.array(Image.open('/kaggle/input/augmenteddataset/augmentfiles/'+img).convert('RGB').resize((128, 128), Image.ANTIALIAS)) for img in dir_list])


  0%|          | 0/1697 [00:00<?, ?it/s]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 4 dimension(s)

In [30]:
arr.shape

(3394, 128, 128, 3)

In [ ]:
np.save('augmented.npy', arr)

# End